# Draft analysis 

---

Group name: Gruppe D

---


## Introduction

*This section includes an introduction to the project motivation, data, and research question. Include a data dictionary* 

In dieser Untersuchung setzt sich die Projektgruppe D mit dem Thema Mobilität außeinander. Dieses Thema ist im Hinblick auf aktuelle gesellschaftliche und politische Problemstellungen bezüglich Klimaerwärmung und Klimazielen sehr relevant. Zürich ist neben Wien eine der am besten ausgebauten Städte bezogen auf den öffentlichen Personenverkehr. Es liegt eine Umfangreiche Datensammlung der Verkehrsbezüge Zürich vor, mit der das bestehende Verkersangebot näher betrachtet wird.
Es stellt sich die Frage, ob eine Korrelation zwischen der Auslastung des öffentlichen Personenverkehrs und den Verspätungen besteht. Daraus sollen Handlungsempfehlunge zu bestimmten Auslastungszeiten abgeleitet werden. 

Zur Analyse der Daten stehen wöchentliche csv-Dateien zur Verfügung, welche eine Vielzahl an Daten beinhalten. Jeder Eintrag stellt einen geplanten Halt eines Fahrzeugs der Verkehrsbetriebe Zürich dar und gibt eine präzise Auskunft über Datum, Linie, Fahrzeug, Haltestelle und vielem mehr. Somit ermöglicht der Datensatz einen Vergleich zwischen den geplanten und tatsächlich erfolgten Fahrzeiten der Fahrzeuge der Verkehrsbetriebe Zürich. Da vielen Daten aus der Tabelle eine interne ID und somit einen Fremdschlüssel zu anderen Tabellen darstellen, sind weitere Tabellen vorhanden, um diese Daten mit konkreten, verständlichen Bezeichnungen verknüpfen zu können.

Der Datenlieferant ist Verkehrsbetriebe Zürich, Departement der Industriellen Betriebe. Die Daten wurden erstmals 2015 erfasst und werden seither wöchentlich sonntags aktualisiert. Sämtliche Zeiten werden dabei auf die Sekunde gemessen. Über die genaue Bereitstellung der Daten gibt es keine Informationen. Da die Daten wöchentlich aktualisiert werden, ist davon auszugehen, dass die Daten durch einen automatischen Export aus einer oder mehreren weiteren Datenbanktabellen zusammengetragen und miteinander verknüpft werden.

Die zur Verfügung stehenden Datensätze sind sehr groß: Eine grobe Kalkulation ergab eine Anzahl von ca. 147 Mio. Datensätzen für einen Zeitraum von zwei Jahren. Aus diesem Grund erscheint es als sinnvoll, die Daten für die Auswertung herunterzubrechen. Dementsprechend werden jeweils die ersten Februarwochen der Jahre 2019 und 2021 miteinander verglichen. Das primäre Ziel des Vergleichs zwischen der Vor-Corona und der Während-Corona-Zeit findet seinen Ursprung in den fehlenden eindeutigen Indikatoren im Datensatz, welche Auskunft über die tatsächliche Auslastung der Fahrzeuge geben. Im Rahmen einer Analyse des Verkehrsverbundes Nürnberg konnte allerdings festgestellt werden, dass die Nutzung öffentlicher Verkehrsmittel gegenüber der Vor-Corona-Phase stark gesunken ist - teilweise um 45%. Unter der Annahme, dass sich dieses Phänomen ebenfalls auf die Stadt Zürich abbilden lässt, ist eine Grundlage für die Analyse geschaffen. Eine weitere Konzentration auf ein oder zwei spezifische Linien (z.B. eine Bus- und eine Tramlinie) ist nicht ausgeschlossen.

## Setup

In [3]:
import pandas as pd
import altair as alt

## Data

### Import data

In [4]:
# import csvs

# 1. Februarwoche 2019
df_19 = pd.read_csv("https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2019/download/Fahrzeiten_SOLL_IST_20190203_20190209.csv")

# 1. Februarwoche 2021
df_21 = pd.read_csv("https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2021/download/Fahrzeiten_SOLL_IST_20210131_20210206.csv")

# Haltestellen
df_haltestellen = pd.read_csv('https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2019/download/Haltestelle.csv')

# Haltepunkte
df_haltepunkte = pd.read_csv('https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2019/download/Haltepunkt.csv')

# Linien
df_linie = pd.read_csv('https://data.stadt-zuerich.ch/dataset/vbz_fahrgastzahlen_ogd/download/LINIE.csv', delimiter=';')


### Data structure

In [44]:
df_19_new = df_19.drop(['kurs', 'seq_von', 'datum_von', 'halt_diva_von','halt_kurz_von1', 'halt_punkt_diva_von','seq_nach', 'halt_diva_nach', 'halt_punkt_diva_nach', 'halt_kurz_nach1', 'datum_nach', 'soll_an_nach','ist_an_nach1', 'soll_ab_nach', 'ist_ab_nach', 'fahrt_id', 'fahrweg_id', 'fw_no', 'fw_typ', 'fw_kurz', 'fw_lang', 'umlauf_von'], axis=1)

df_19_new.head()

,linie,richtung,betriebsdatum,fahrzeug,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach
0,2,1,03.02.19,2072,17580,17604,17580,17604,2251,1906,47032,10563
1,2,1,03.02.19,3033,17802,17820,17802,17820,1306,1502,44135,44530
2,2,1,03.02.19,3033,18222,18186,18234,18199,2228,2590,49163,46896
3,2,1,03.02.19,3033,18156,18124,18174,18138,1528,2228,44152,49163
4,2,1,03.02.19,3033,18054,18017,18066,18038,2657,1528,10574,44152


### Data corrections

In [45]:
df_haltestellen_new = df_haltestellen.drop(['halt_diva', 'halt_kurz', 'halt_ist_aktiv'], axis=1)
df_haltestellen_new.head()

,halt_id,halt_lang
0,143,"Birmensdorf ZH, Sternen/WSL"
1,309,"Waldegg, Birmensdorferstrasse"
2,373,"Zürich Flughafen, Fracht"
3,539,"Thalwil, Bahnhof"
4,588,"Zürich Flughafen, Bahnhof"


In [46]:
df_all = df_19_new.join(df_haltestellen_new.set_index('halt_id'), on='halt_id_von')
df_all = df_all.rename(columns={"halt_lang": "halt_lang_von"})

df_all = df_all.join(df_haltestellen_new.set_index('halt_id'), on='halt_id_nach')
df_all = df_all.rename(columns={"halt_lang": "halt_lang_nach"})

df_all.head()

,linie,richtung,betriebsdatum,fahrzeug,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_lang_von,halt_lang_nach
0,2,1,03.02.19,2072,17580,17604,17580,17604,2251,1906,47032,10563,"Zürich, Depot 4 Elisabethenstr","Zürich,Kalkbreite/Bhf.Wiedikon"
1,2,1,03.02.19,3033,17802,17820,17802,17820,1306,1502,44135,44530,"Zürich, Bezirksgebäude","Zürich, Stauffacher"
2,2,1,03.02.19,3033,18222,18186,18234,18199,2228,2590,49163,46896,"Zürich, Kantonalbank","Zürich, Bürkliplatz"
3,2,1,03.02.19,3033,18156,18124,18174,18138,1528,2228,44152,49163,"Zürich, Paradeplatz","Zürich, Kantonalbank"
4,2,1,03.02.19,3033,18054,18017,18066,18038,2657,1528,10574,44152,"Zürich, Sihlstrasse","Zürich, Paradeplatz"


## Analysis

### Descriptive statistics

### Exploratory data analysis

## Visualizations

### Visualization ideas

### Save Visualizations



Save your draft visualizations in the folder `reports/visualizations/`. Use a meaningful name (always include the word `draft` and a `timestamp`in your filename).

## Conclusion and recommended action